In [16]:
import tensorflow as tf
import keras

In [17]:
model = keras.saving.load_model("weights-best.weights.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 3)              │             7 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_1 (Normalization) │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,119 (27.82 KB)

 Trainable params: 2,369 (9.25 KB)

 Non-trainable params: 10 (48.00 B)

 Optimizer params: 4,740 (18.52 KB)

In [21]:
input_length = model.input_shape[1]
dense_layers_info = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        num_neurons = layer.units
        dense_layers_info.append((num_neurons))

num_dense_layers = len(dense_layers_info)
print("dense layer number:", num_dense_layers)
print("neuron number:", dense_layers_info)

dense layer number: 3
neuron number: [64, 32, 1]


In [22]:
with open('variable.txt', 'w') as f:
    f.write(f'''FUNCTION_BLOCK FB_NN_Forward_Propagation
// calculation the forward propagation of neural network
// include normalization layer and denormalization layer
VAR_INPUT
	// input : [x, tau, v] at time k
	input : ARRAY[0..{input_length-1}] OF LREAL;
END_VAR
VAR_OUTPUT
	// output [delta_v] at time k+1
	output : LREAL;
END_VAR
VAR
	// activation function
    activation : FB_Relu;
	// pre-allocation of layer outputs''')
    for i in range(num_dense_layers-1):
        f.write(f'''\n\tHiddenLayer{i+1}_output : ARRAY[0..{dense_layers_info[i]-1}] OF LREAL;''')
    f.write(f'''\n\t// input normalizer
	normalizer : FB_Normalization_Input;
	// output denormalizer
	denormalizer : FB_Denormalization_Output;
	// normalized input
	input_norm : ARRAY[0..{input_length-1}] OF LREAL;	
	// normalized output
	output_norm : LREAL;
	// counters''')
    for idx in range(num_dense_layers-1):
        letter = chr(ord('i') + idx)
        declaration = f"\n\t{letter} : INT;"
        f.write(declaration)
    f.write(f'''\nEND_VAR''')

In [23]:
with open('main.txt', 'w') as f:
    f.write(f'''// input normalization layer
normalizer(input:=input, input_norm=>input_norm);

// first hidden layer
FOR i:=0 TO {dense_layers_info[0]-1} DO
	HiddenLayer1_output[i] := 0;
	FOR j := 0 TO {input_length-1} DO
		HiddenLayer1_output[i] := HiddenLayer1_output[i] + input_norm[j]*GVL_Param_NN.HiddenLayer1_Weight[i,j];
	END_FOR
	HiddenLayer1_output[i] := HiddenLayer1_output[i] + GVL_Param_NN.HiddenLayer1_Bias[i];
	activation(x:= HiddenLayer1_output[i], result => HiddenLayer1_output[i]);
END_FOR

// second hidden layer''')
    
    for i in range(1, num_dense_layers-1):
        f.write(f'''\nFOR i:=0 TO {dense_layers_info[i]-1} DO
    HiddenLayer{i+1}_output[i] := 0;
    FOR j := 0 TO {dense_layers_info[i-1]-1} DO
        HiddenLayer{i+1}_output[i] := HiddenLayer{i+1}_output[i] + HiddenLayer{i}_output[j]*GVL_Param_NN.HiddenLayer{i+1}_Weight[i,j];
    END_FOR
    HiddenLayer{i+1}_output[i] := HiddenLayer{i+1}_output[i] + GVL_Param_NN.HiddenLayer{i+1}_Bias[i];
    activation(x:= HiddenLayer{i+1}_output[i], result => HiddenLayer{i+1}_output[i]);
END_FOR

// linear output layer''')
        
    f.write(f'''\noutput_norm := 0;
FOR j := 0 TO {dense_layers_info[-2]-1} DO
    output_norm := output_norm + HiddenLayer{num_dense_layers-1}_output[j]* GVL_Param_NN.OutputLayer_Weight[0,j];
END_FOR
output_norm := output_norm + GVL_Param_NN.OutputLayer_Bias[0];

// output (de-)normalization layer
denormalizer(output_norm :=output_norm, output => output);''')